# Statistical test of Treatment A & B

This is the analysis of Treatment A & B for the novo case for presentation on 2022-03-28 @ 13.00



In [5]:
# install and import important modules
import sys


!{sys.executable} -m pip install numpy pandas matplotlib

print('prereq modules installed')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

print('imported modules')

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 13.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.5/899.5 KB 14.5 MB/s eta 0:00:0031m20.6 MB/s eta 0:00:01
prereq modules installed
imported modules


# Clean up of data

This section entails the cleanup of data. 
To avoid data curruption, The cleaned up data is saved into two new sheet in in the file 'cleaned-data.xlsx'.

As this is a case study for the purpose of describing the workflow, I will be 
